In [3]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, shape
import folium
import os

In [2]:
!ls "data/pedestrian"

pedestrian-network.geojson  pedestrian-network.xls
pedestrian-network.json     pedestrian-network_short.xlsx
pedestrian-network.parquet


In [4]:
geo_folder = "data/pedestrian"
geo_path = os.path.join(geo_folder, "pedestrian-network_short.xlsx")

In [6]:
df = pd.read_excel(geo_path)
df.head()

,geo_point_2d,geo_shape,objectid,neworkid
0,"-37.80611422595, 144.96314903935001","{""coordinates"": [[144.9631440326, -37.80614306...",15738,NaN
1,"-37.80605319185, 144.96544271714998","{""coordinates"": [[144.9653585112, -37.80604322...",15742,NaN
2,"-37.8057452443, 144.9420012438","{""coordinates"": [[144.9419916293, -37.80579960...",15746,NaN
3,"-37.8059843314, 144.9580393526","{""coordinates"": [[144.9580973857, -37.80599241...",15747,NaN
4,"-37.805963458849995, 144.95786965805002","{""coordinates"": [[144.9579247422, -37.80596977...",15749,NaN


In [9]:
df.dtypes

geo_point_2d     object
geo_shape        object
objectid          int64
neworkid        float64
dtype: object

In [7]:
# Step 1: Convert geo_point_2d to Latitude & Longitude
def extract_lat_lon(geo_point):
    try:
        lat, lon = map(float, geo_point.split(", "))
        return lat, lon
    except:
        return None, None

In [8]:
# Step 2: Parse geo_shape column
def parse_geo_shape(shape_str):
    try:
        shape_dict = json.loads(shape_str.replace("'", '"'))  # Ensure valid JSON
        return shape(shape_dict)  # Convert to Shapely geometry
    except:
        return None


In [10]:
df['latitude'], df['longitude'] = zip(*df['geo_point_2d'].apply(extract_lat_lon))

In [11]:
df['geometry'] = df['geo_shape'].apply(parse_geo_shape)

In [12]:
df.head()

,geo_point_2d,geo_shape,objectid,neworkid,latitude,longitude,geometry
0,"-37.80611422595, 144.96314903935001","{""coordinates"": [[144.9631440326, -37.80614306...",15738,NaN,-37.806114,144.963149,"LINESTRING (144.9631440326 -37.8061430613, 144..."
1,"-37.80605319185, 144.96544271714998","{""coordinates"": [[144.9653585112, -37.80604322...",15742,NaN,-37.806053,144.965443,"LINESTRING (144.9653585112 -37.8060432222, 144..."
2,"-37.8057452443, 144.9420012438","{""coordinates"": [[144.9419916293, -37.80579960...",15746,NaN,-37.805745,144.942001,"LINESTRING (144.9419916293 -37.8057996099, 144..."
3,"-37.8059843314, 144.9580393526","{""coordinates"": [[144.9580973857, -37.80599241...",15747,NaN,-37.805984,144.958039,"LINESTRING (144.9580973857 -37.8059924102, 144..."
4,"-37.805963458849995, 144.95786965805002","{""coordinates"": [[144.9579247422, -37.80596977...",15749,NaN,-37.805963,144.957870,"LINESTRING (144.9579247422 -37.8059697752, 144..."


In [13]:
# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
gdf.head()

,geo_point_2d,geo_shape,objectid,neworkid,latitude,longitude,geometry
0,"-37.80611422595, 144.96314903935001","{""coordinates"": [[144.9631440326, -37.80614306...",15738,NaN,-37.806114,144.963149,"LINESTRING (144.96314 -37.80614, 144.96315 -37..."
1,"-37.80605319185, 144.96544271714998","{""coordinates"": [[144.9653585112, -37.80604322...",15742,NaN,-37.806053,144.965443,"LINESTRING (144.96536 -37.80604, 144.96553 -37..."
2,"-37.8057452443, 144.9420012438","{""coordinates"": [[144.9419916293, -37.80579960...",15746,NaN,-37.805745,144.942001,"LINESTRING (144.94199 -37.8058, 144.94201 -37...."
3,"-37.8059843314, 144.9580393526","{""coordinates"": [[144.9580973857, -37.80599241...",15747,NaN,-37.805984,144.958039,"LINESTRING (144.9581 -37.80599, 144.95798 -37...."
4,"-37.805963458849995, 144.95786965805002","{""coordinates"": [[144.9579247422, -37.80596977...",15749,NaN,-37.805963,144.957870,"LINESTRING (144.95792 -37.80597, 144.95781 -37..."


In [14]:
# Step 3: Visualization using folium
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=14)
for _, row in df.iterrows():
    if not pd.isnull(row['latitude']) and not pd.isnull(row['longitude']):
        folium.Marker([row['latitude'], row['longitude']], popup=f"ID: {row['objectid']}").add_to(m)

In [15]:
m.save("pedestrian_network_map.html")

這個html沒辦法顯示任何內容。